In [7]:
import json
import os
import numpy as np

def load_existing_results(file_path="forecasting_results.json"):
    """
    Load existing results from a JSON file.
    Returns an empty dictionary if the file doesn't exist.
    """
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return json.load(f)
    return {}


def save_results_to_json(data, file_path="forecasting_results.json"):
    """
    Save the results dictionary to a JSON file, handling NumPy data types.
    """

    # Handle NumPy data types (recursive conversion)
    def convert_numpy(obj):
        if isinstance(obj, dict):
            return {k: convert_numpy(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(i) for i in obj]
        elif isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert arrays to lists
        else:
            return obj

    # Convert data and save to JSON
    data = convert_numpy(data)
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"✅ Results saved to {file_path}")



def store_results(dataset_name, horizons, horizon_value, experiment_type, backbone, mae_result, file_path="forecasting_results.json"):
    """
    Store MAE results for a given experiment type (stl_mae, mtl_mae, global_mae) per horizon.

    Args:
    - dataset_name (str): Name of the dataset (e.g., 'Solar', 'Air Quality').
    - horizons (list): List of horizon values (e.g., [1, 2, 4, 8, 16]).
    - horizon_value (int): The horizon corresponding to the mae_result provided.
    - experiment_type (str): One of ['stl_mae', 'mtl_mae', 'global_mae'].
    - backbone (str): Model backbone name (e.g., 'Deep_LSTM', 'simple_transformer').
    - mae_result (list): MAE values for the current horizon (list of floats).
    - file_path (str): JSON file to store the results.

    Returns:
    - None
    """
    # Load existing results
    results_dict = load_existing_results(file_path)

    # Create dataset entry if it doesn't exist
    dataset_key = f"{dataset_name}_{backbone}"
    if dataset_key not in results_dict:
        results_dict[dataset_key] = {
            "horizons": horizons,
            "mtl": [[] for _ in horizons],
            "global": [[] for _ in horizons],
            "independent": [[] for _ in horizons]
        }

    # Find index for the given horizon
    try:
        horizon_index = horizons.index(horizon_value)
    except ValueError:
        raise ValueError(f"⚠️ Horizon value {horizon_value} not found in {horizons}.")

    # Append the mae_result to the correct horizon
    results_dict[dataset_key][experiment_type][horizon_index].extend(mae_result)

    # Save updated results
    save_results_to_json(results_dict, file_path)

In [131]:

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import dateutil.parser
import matplotlib.pyplot as plt

def df_to_X_y(df, features, target, window_size=32, horizon=1):
    if target not in features:
        features = [target] + features

    data = df[features].to_numpy()
    target_data = df[target].to_numpy()

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i + window_size])
        y.append(target_data[i + window_size: i + window_size + horizon])

    return np.array(X), np.array(y)


def load_and_preprocess_site_data(site_path,features, target ,window_size=32, horizon=16, min_date=None, max_date=None, batch_size=16, device='cpu'):
    """
    Loads and preprocesses time series data for a given site.

    Args:
    - site_path (str): Path to the site CSV file.
    - window_size (int): Number of past time steps for input.
    - horizon (int): Number of future steps to predict.

    Returns:
    - train_loader, val_loader, test_loader: DataLoaders for training, validation, and testing.
    """
    df = pd.read_csv(site_path)
    
    # Convert date column to datetime if it exists
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

        # Filter data between min_date and max_date
        if min_date:
            min_date = dateutil.parser.parse(min_date) if isinstance(min_date, str) else min_date
            df = df[df['date'] >= min_date]

        if max_date:
            max_date = dateutil.parser.parse(max_date) if isinstance(max_date, str) else max_date
            df = df[df['date'] <= max_date]

        # Drop the date column after filtering
        df.drop(columns=['date'], inplace=True)
        
    
    # Perform an 80-20 split based on time order
    train_size = int(0.8 * len(df))
    train_df = df.iloc[:train_size]  # 80% for training & validation
    test_df = df.iloc[train_size:]   # 20% for final testing (future unseen data)

    # Split train_df further into Train (80%) and Validation (20%)
    # val_size = int(0.2 * len(train_df))  # 16% of full dataset
    # train_df, val_df = train_df.iloc[:-val_size], train_df.iloc[-val_size:]

    print(f"Train size: {len(train_df)} | Validation size: {len([])} | Test size: {len(test_df)}")

    # Standardize each separately to prevent data leakage
    train_mean, train_std = train_df.mean(), train_df.std()
    
    train_df = (train_df - train_mean) / (train_std + 1e-8)
    # val_df = (val_df - train_mean) / (train_std + 1e-8)  # Normalize validation using train stats
    test_df = (test_df - train_mean) / (train_std + 1e-8)  # Normalize test using train stats

    # Convert DataFrame to NumPy arrays for LSTM
    X_train, y_train = df_to_X_y(train_df,features, target, window_size, horizon)
    # X_val, y_val = df_to_X_y(val_df, features, target, window_size, horizon)
    X_test, y_test = df_to_X_y(test_df,features, target, window_size, horizon)

    # Convert to PyTorch tensors
    # train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
    # val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
    # test_data = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
    
    train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device), torch.tensor(y_train, dtype=torch.float32).to(device))
    # val_data = TensorDataset(torch.tensor(X_val, dtype=torch.float32).to(device), torch.tensor(y_val, dtype=torch.float32).to(device))
    test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32).to(device), torch.tensor(y_test, dtype=torch.float32).to(device))

    # Create DataLoaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
    # val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, drop_last=True)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=True)

    # return train_loader, val_loader, test_loader
    return train_loader, test_loader

# ---------- Data Loader Function with Detrending ----------
def load_and_preprocess_site_data_detrend(site_path, features, target, window_size=32, horizon=1, 
                                          min_date=None, max_date=None, batch_size=16, device='cpu'):
    df = pd.read_csv(site_path)

    # Convert date column to datetime if it exists
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

        # Filter data between min_date and max_date
        if min_date:
            min_date = dateutil.parser.parse(min_date) if isinstance(min_date, str) else min_date
            df = df[df['date'] >= min_date]

        if max_date:
            max_date = dateutil.parser.parse(max_date) if isinstance(max_date, str) else max_date
            df = df[df['date'] <= max_date]

        # Drop the date column after filtering
        df.drop(columns=['date'], inplace=True)
        
    if target not in features:
        features = [target] + features

    all_columns = features
    if not all(col in df.columns for col in all_columns):
        missing = [col for col in all_columns if col not in df.columns]
        raise ValueError(f"Missing columns in dataset: {missing}")

    train_size = int(0.8 * len(df))
    train_df = df.iloc[:train_size].copy()
    test_df = df.iloc[train_size:].copy()

    print(f"Train size: {len(train_df)} | Test size: {len(test_df)}")

    # ------------------- Detrending Step -------------------
    degree = 2  # Polynomial degree for detrending (can be tuned)
    
    train_idx = np.arange(len(train_df))
    test_idx = np.arange(len(test_df)) + len(train_df)

    # Center indices to prevent floating-point instability in polynomial fitting
    train_idx_centered = train_idx - train_idx.mean()
    test_idx_centered = test_idx - train_idx.mean()

    # Fit polynomial trend using training data for both the target and features
    poly_dict = {}  # Store polynomial trends for all columns

    for col in features:
        coeffs = np.polyfit(train_idx_centered, train_df[col].values, degree)
        poly = np.poly1d(coeffs)
        poly_dict[col] = poly

        # Remove trend
        train_df[col] -= poly(train_idx_centered)
        test_df[col] -= poly(test_idx_centered)
    # -------------------------------------------------------

    # Normalize features using training statistics
    train_mean, train_std = train_df[all_columns].mean(), train_df[all_columns].std()
    train_df[all_columns] = (train_df[all_columns] - train_mean) / (train_std + 1e-8)
    test_df[all_columns] = (test_df[all_columns] - train_mean) / (train_std + 1e-8)

    # Prepare sequences for training/testing
    X_train, y_train = df_to_X_y(train_df, features, target, window_size, horizon)
    X_test, y_test = df_to_X_y(test_df, features, target, window_size, horizon)

    # Convert to PyTorch tensors
    train_data = TensorDataset(
        torch.tensor(X_train, dtype=torch.float32).to(device),
        torch.tensor(y_train, dtype=torch.float32).to(device)
        )
    test_data = TensorDataset(
        torch.tensor(X_test, dtype=torch.float32).to(device),
        torch.tensor(y_test, dtype=torch.float32).to(device)
        )

    # Create DataLoaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=True)

    return train_loader, test_loader, poly_dict  # Return polynomial trends for re-trending



In [117]:
# --------------------------- #
#          LSTM Model         #
# --------------------------- #

class LSTMModel(nn.Module):
    def __init__(self, num_features, time_window, output_window, num_labels, num_layers=2, hidden_size=16, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.output_window = output_window
        self.num_labels = num_labels

        self.lstm = nn.LSTM(num_features,
                            hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, num_labels * output_window)

    def forward(self, x):
        B, T, D = x.shape  
        x_, _ = self.lstm(x)
        last_hidden = x_[:, -1, :]  # Take last time step's hidden state
        x_ = self.fc(last_hidden)
        x_ = x_.reshape(B, self.output_window, self.num_labels)
        return x_


In [118]:
# --------------------------- #
#      Training Function      #
# --------------------------- #

def train_model(model, train_loader, val_loader, num_epochs=30, lr=1e-3, wd=1e-5):
    """
    Trains an LSTM model with validation.

    Args:
    - model (nn.Module): The LSTM model.
    - train_loader (DataLoader): DataLoader for training.
    - val_loader (DataLoader): DataLoader for validation.
    - num_epochs (int): Number of training epochs.
    - lr (float): Learning rate.

    Returns:
    - None
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=wd)
    device = "cuda"

    model.to(device)
    
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        # print(f"\nEPOCH {epoch+1}")
        
        # -------------------- Training -------------------- #
        model.train()
        running_train_loss = 0.0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            output = model(batch_x)
            loss = criterion(output, batch_y.unsqueeze(-1))  # Ensure correct shape
            running_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = running_train_loss / len(train_loader)
        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # -------------------- Validation -------------------- #
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():  # Disable gradient calculations
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                output = model(batch_x)
                loss = criterion(output, batch_y.unsqueeze(-1))  # Ensure correct shape
                running_val_loss += loss.item()

        avg_val_loss = running_val_loss / len(val_loader)
        avg_val_loss = running_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)


        # Print losses for the epoch
        # print(f"Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

    print("Training complete!")
    
# --------------------------- #
#      Evaluation Function    #
# --------------------------- #

def evaluate_model(model, test_loader):
    """
    Evaluates the LSTM model on test data.

    Args:
    - model (nn.Module): Trained LSTM model.
    - test_loader (DataLoader): DataLoader for testing.

    Returns:
    - mean_mae (float): Mean Absolute Error.
    """
    model.eval()
    criterion = nn.L1Loss(reduction='mean')
    mae_list = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")
            predictions = model(batch_x)
            mae = criterion(predictions, batch_y.unsqueeze(-1)).item()
            mae_list.append(mae)

    return np.mean(mae_list)

In [119]:
import os
import torch
import numpy as np
from torch.optim.lr_scheduler import ExponentialLR

def run_experiment_mae_independent_lstm():
    """
    Runs independent LSTM experiments for multiple datasets and horizons.
    Appends results to output.txt with dataset names, site-wise MAEs, and horizon details.
    """
    datasets = [
        {
            'name': 'Air Quality',
            'directory': "../processed_ds/air_quality_cluster/",
            'features': ['PM2.5', 'OT', 'PM10', 'NO2'],
            'target': 'PM2.5',
            'min_date': "2014-09-01",
            'max_date': "2014-11-12 19:00",
            'num_features': 4
        },
        # {
        #     'name': 'Solar',
        #     'directory': "../processed_ds/solar/",
        #     'features': ['loc-1', 'loc-2', 'loc-3', 'loc-4'],
        #     'target': 'loc-1',
        #     'min_date': "2006-09-01",
        #     'max_date': "2006-09-08 4:50",
        #     'num_features': 4
        # },
        # {
        #     'name': 'Crypto',
        #     'directory': "../processed_ds/crypto-data/",
        #     'features': ['Open', 'High', 'Low', 'OT', 'Volume'],
        #     'target': 'OT',
        #     'min_date': "2018-04-01",
        #     'max_date': "2018-06-15",
        #     'num_features': 5
        # },
        # {
        #     'name': 'Sales',
        #     'directory': "../processed_ds/stores_data/",
        #     'min_date': "2013-01-16",
        #     'max_date': "2015-07-31",
        #     'num_features': 7
        # }
    ]

    horizons = [1,2,4,8,16]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seq_len = 32
    batch_size = 32
    num_epochs = 30

    for dataset in datasets:
        site_files = [
            os.path.join(dataset['directory'], f, f"{f}.csv")
            for f in os.listdir(dataset['directory'])
            if os.path.isdir(os.path.join(dataset['directory'], f))
        ]

        for horizon in horizons:
            site_mae_list = []
            print(f"\n==================== Dataset: {dataset['name']} | Horizon: {horizon} ====================")

            for site_path in site_files[0:30]:
                print(f"\nProcessing Site: {site_path}")

                # Load data
                train_loader, test_loader = load_and_preprocess_site_data(
                    site_path,
                    features=dataset['features'],
                    target=dataset['target'],
                    horizon=horizon,
                    min_date=dataset['min_date'],
                    max_date=dataset['max_date'],
                    batch_size=batch_size,
                    device=device
                )

                # LSTM Model Definition
                model = LSTMModel(
                    num_features=dataset['num_features'],
                    time_window=32,
                    output_window=horizon,
                    num_labels=1,
                    num_layers=2,
                    hidden_size=16,
                    dropout=0.5
                )
                total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                print("Total trainable parameters:", total_params)
                model.to(device)

                # Training setup
                optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
                scheduler = ExponentialLR(optimizer, gamma=0.95)
                criterion = torch.nn.MSELoss()

                # Training loop
                model.train()
                for epoch in range(num_epochs):
                    epoch_train_losses = []
                    for batch_x, batch_y in train_loader:
                        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                        optimizer.zero_grad()
                        output = model(batch_x)
                        loss = criterion(output, batch_y.unsqueeze(-1))
                        loss.backward()
                        optimizer.step()
                        epoch_train_losses.append(loss.item())
                    scheduler.step()
                    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {np.mean(epoch_train_losses):.4f}")

                # Evaluation
                model.eval()
                mae_criterion = torch.nn.L1Loss()
                test_preds, test_targets = [], []
                with torch.no_grad():
                    for batch_x, batch_y in test_loader:
                        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                        preds = model(batch_x)
                        test_preds.append(preds.cpu())
                        test_targets.append(batch_y.unsqueeze(-1).cpu())
                    
                test_preds = torch.cat(test_preds, dim=0)
                test_targets = torch.cat(test_targets, dim=0)
                
            
                test_mae = mae_criterion(test_preds, test_targets)
                print(f"Site: {site_path}, Test MAE: {test_mae.item():.4f}")
                site_mae_list.append(test_mae.item())

            # Average MAE across all sites for this horizon
            avg_mae = np.mean(site_mae_list)
            print(f"\nAverage MAE for {dataset['name']} at horizon {horizon}: {avg_mae:.4f}")

            # Append results to output.txt
            with open("output_test.txt", "a") as f:
                f.write("\n==================== LSTM INDEPENDENT FORECASTING MODEL RESULTS ====================\n")
                f.write(f"Dataset: {dataset['name']}\n")
                f.write(f"Horizon: {horizon}\n")
                f.write(f"MAE per site: {site_mae_list}\n")
                f.write(f"Mean MAE: {avg_mae:.4f}\n")
            # TODO: FIX THIS before storing results.
            # store_results(
            #     dataset_name=dataset['name'],
            #     horizons=[1,2,4,8,16],
            #     experiment_type='independent',
            #     mae_result=site_mae_list,
            #     backbone='simple_lstm',
            #     horizon_value=horizon
            # )

    print("\n🏆 All independent LSTM Forecasting Model experiments are complete! 🏆")


In [134]:
from sklearn.metrics import mean_absolute_error
def run_experiment_mae_independent_lstm_detrend():
    """
    Runs independent LSTM experiments for multiple datasets and horizons.
    Appends results to output.txt with dataset names, site-wise MAEs, and horizon details.
    """
    datasets = [
        # {
        #     'name': 'Air Quality',
        #     'directory': "../processed_ds/air_quality_cluster/",
        #     'features': ['PM2.5', 'OT', 'PM10', 'NO2'],
        #     'target': 'PM2.5',
        #     'min_date': "2014-09-01",
        #     'max_date': "2014-11-12 19:00",
        #     'num_features': 4
        # },
        {
            'name': 'Solar',
            'directory': "../processed_ds/solar/",
            'features': ['loc-1', 'loc-2', 'loc-3', 'loc-4'],
            'target': 'loc-1',
            'min_date': "2006-09-01",
            'max_date': "2006-09-08 4:50",
            'num_features': 4
        },
        # {
        #     'name': 'Crypto',
        #     'directory': "../processed_ds/crypto-data/",
        #     'features': ['Open', 'High', 'Low', 'OT', 'Volume'],
        #     'target': 'OT',
        #     'min_date': "2018-04-01",
        #     'max_date': "2018-06-15",
        #     'num_features': 5
        # },
        # {
        #     'name': 'Sales',
        #     'directory': "../processed_ds/stores_data/",
        #     'min_date': "2013-01-16",
        #     'max_date': "2015-07-31",
        #     'num_features': 7
        # }
    ]

    horizons = [1,2,4,8,16]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seq_len = 32
    batch_size = 16
    num_epochs = 40

    for dataset in datasets:
        site_files = [
            os.path.join(dataset['directory'], f, f"{f}.csv")
            for f in os.listdir(dataset['directory'])
            if os.path.isdir(os.path.join(dataset['directory'], f))
        ]

        for horizon in horizons:
            site_mae_list = []
            print(f"\n==================== Dataset: {dataset['name']} | Horizon: {horizon} ====================")

            for site_path in site_files[0:30]:
                print(f"\nProcessing Site: {site_path}")

                # Load data
                train_loader, test_loader, poly_dict = load_and_preprocess_site_data_detrend(
                    site_path,
                    features=dataset['features'],
                    target=dataset['target'],
                    horizon=horizon,
                    min_date=dataset['min_date'],
                    max_date=dataset['max_date'],
                    batch_size=batch_size,
                    device=device
                )

                # LSTM Model Definition
                model = LSTMModel(
                    num_features=dataset['num_features'],
                    time_window=32,
                    output_window=horizon,
                    num_labels=1,
                    num_layers=2,
                    hidden_size=16,
                    dropout=0.5
                )
                total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                print("Total trainable parameters:", total_params)
                model.to(device)

                # Training setup
                optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
                scheduler = ExponentialLR(optimizer, gamma=0.95)
                criterion = torch.nn.MSELoss()

                # Training loop
                model.train()
                for epoch in range(num_epochs):
                    epoch_train_losses = []
                    for batch_x, batch_y in train_loader:
                        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                        optimizer.zero_grad()
                        output = model(batch_x)
                        loss = criterion(output, batch_y.unsqueeze(-1))
                        loss.backward()
                        optimizer.step()
                        epoch_train_losses.append(loss.item())
                    scheduler.step()
                    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {np.mean(epoch_train_losses):.4f}")

                # Evaluation
                model.eval()
                mae_criterion = torch.nn.L1Loss()
                test_preds, test_targets, test_indices, test_features = [], [], [], []

                with torch.no_grad():
                    for i, (batch_x, batch_y) in enumerate(test_loader):
                        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                        preds = model(batch_x)
                        
                        test_preds.append(preds.cpu())
                        test_targets.append(batch_y.unsqueeze(-1).cpu())
                        test_features.append(batch_x.cpu())

                        # Store test indices for inverse detrending
                        start_idx = i * batch_size
                        end_idx = start_idx + len(batch_x)
                        test_indices.extend(range(start_idx, end_idx))

                # Convert lists to tensors and remove extra dimensions
                test_preds = torch.cat(test_preds, dim=0).squeeze().numpy()  # Shape: (N, horizon)
                test_targets = torch.cat(test_targets, dim=0).squeeze().numpy()  # Shape: (N, horizon)
                test_features = torch.cat(test_features, dim=0).numpy()  # Shape: (N, seq_len, num_features)
                
                if test_preds.ndim == 1:
                    test_preds = np.expand_dims(test_preds, axis=1)  # Shape: (N, 1)
                if test_targets.ndim == 1:
                    test_targets = np.expand_dims(test_targets, axis=1)  # Shape: (N, 1)
                
                # ✅ Ensure test_indices has correct length
                test_indices = np.array(test_indices[:len(test_preds)])

                # ✅ **Apply inverse detrending (Add trend back to predictions, targets, and features)**
                for col_idx, col_name in enumerate(['loc-1', 'loc-2', 'loc-3', 'loc-4']):
                    trend_values = poly_dict[col_name](test_indices)  # Shape: (N,)
                    if col_name == 'loc-1':
                        # Expand trend across forecast horizons for target
                        trend_values = np.expand_dims(trend_values, axis=1)  # Shape: (N, 1)
                        if test_preds.shape[1] > 1:  # Only tile if horizon > 1
                            trend_values = np.tile(trend_values, (1, test_preds.shape[1]))  # Expand across horizon
                        test_preds += trend_values
                        test_targets += trend_values
                    else:
                        # Apply to each feature in test_features
                        # test_features[:, :, col_idx] += np.expand_dims(trend_values, axis=1)  # Shape: (N, seq_len)
                        # Apply to each feature in test_features
                        trend_values = np.expand_dims(trend_values, axis=1)  # Shape: (N, 1)
                        trend_values = np.tile(trend_values, (1, test_features.shape[1]))  # Expand across seq_len
                        test_features[:, :, col_idx] += trend_values  # Add trend back to each feature
                        
                # ✅ Convert back to PyTorch tensors for MAE calculation
                test_preds = torch.tensor(test_preds, dtype=torch.float32)
                test_targets = torch.tensor(test_targets, dtype=torch.float32)

                # ✅ Compute MAE
                test_mae = mae_criterion(test_preds, test_targets)
                print(f"Site: {site_path}, Test MAE: {test_mae.item():.4f}")
                site_mae_list.append(test_mae.item())

            # Average MAE across all sites for this horizon
            avg_mae = np.mean(site_mae_list)
            print(f"\nAverage MAE for {dataset['name']} at horizon {horizon}: {avg_mae:.4f}")

            # Append results to output.txt
            with open("output_test.txt", "a") as f:
                f.write("\n==================== LSTM INDEPENDENT FORECASTING MODEL RESULTS ====================\n")
                f.write(f"Dataset: {dataset['name']}\n")
                f.write(f"Horizon: {horizon}\n")
                f.write(f"MAE per site: {site_mae_list}\n")
                f.write(f"Mean MAE: {avg_mae:.4f}\n")

    print("\n🏆 All independent LSTM Forecasting Model experiments are complete! 🏆")

In [135]:
run_experiment_mae_independent_lstm()


==================== Dataset: Air Quality | Horizon: 1 ====================

Processing Site: ../processed_ds/air_quality_cluster/site-11/site-11.csv
Train size: 1398 | Validation size: 0 | Test size: 350
Total trainable parameters: 3601
Epoch 1/30 - Train Loss: 1.0192
Epoch 2/30 - Train Loss: 0.9972
Epoch 3/30 - Train Loss: 0.9709
Epoch 4/30 - Train Loss: 0.9251
Epoch 5/30 - Train Loss: 0.9072
Epoch 6/30 - Train Loss: 0.8555
Epoch 7/30 - Train Loss: 0.7962
Epoch 8/30 - Train Loss: 0.7270
Epoch 9/30 - Train Loss: 0.6570
Epoch 10/30 - Train Loss: 0.5977
Epoch 11/30 - Train Loss: 0.5314
Epoch 12/30 - Train Loss: 0.4617
Epoch 13/30 - Train Loss: 0.4261
Epoch 14/30 - Train Loss: 0.3909
Epoch 15/30 - Train Loss: 0.3561
Epoch 16/30 - Train Loss: 0.3391
Epoch 17/30 - Train Loss: 0.3215
Epoch 18/30 - Train Loss: 0.3126
Epoch 19/30 - Train Loss: 0.3023
Epoch 20/30 - Train Loss: 0.2964
Epoch 21/30 - Train Loss: 0.2888
Epoch 22/30 - Train Loss: 0.2819
Epoch 23/30 - Train Loss: 0.2734
Epoch 24/30

KeyboardInterrupt: 

In [136]:
run_experiment_mae_independent_lstm_detrend()


==================== Dataset: Solar | Horizon: 1 ====================

Processing Site: ../processed_ds/solar/solar_al/solar_al.csv
Train size: 1660 | Test size: 415
Total trainable parameters: 3601
Epoch 1/40 - Train Loss: 0.9817
Epoch 2/40 - Train Loss: 0.9180
Epoch 3/40 - Train Loss: 0.8030
Epoch 4/40 - Train Loss: 0.5795
Epoch 5/40 - Train Loss: 0.3672
Epoch 6/40 - Train Loss: 0.2810
Epoch 7/40 - Train Loss: 0.2506
Epoch 8/40 - Train Loss: 0.2419
Epoch 9/40 - Train Loss: 0.2316
Epoch 10/40 - Train Loss: 0.2259
Epoch 11/40 - Train Loss: 0.2218
Epoch 12/40 - Train Loss: 0.2153
Epoch 13/40 - Train Loss: 0.2152
Epoch 14/40 - Train Loss: 0.2064
Epoch 15/40 - Train Loss: 0.2083
Epoch 16/40 - Train Loss: 0.2052
Epoch 17/40 - Train Loss: 0.1982
Epoch 18/40 - Train Loss: 0.1921
Epoch 19/40 - Train Loss: 0.1899
Epoch 20/40 - Train Loss: 0.1919
Epoch 21/40 - Train Loss: 0.1893
Epoch 22/40 - Train Loss: 0.1838
Epoch 23/40 - Train Loss: 0.1832
Epoch 24/40 - Train Loss: 0.1841
Epoch 25/40 - Tra

KeyboardInterrupt: 